In [1]:
import os, sys
import glob
import time
from datetime import timedelta, datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
#setting offilne
plotly.offline.init_notebook_mode(connected=True)
import pyarrow.parquet as pq
import json
from gezi import tqdm
import gezi
import qgrid

In [2]:
def gtime(timestamp):
  x = time.localtime(timestamp)
  return f'{x.tm_mon}-{x.tm_mday}-{x.tm_hour}-{x.tm_min}'

In [3]:
def gday(timestamp):
  x = time.localtime(timestamp)
  return x.tm_mday

In [4]:
dcs = []
for k in tqdm(range(30)):
  d = gezi.read_parquet(f'../input/train/part_{k + 1}/context.parquet')
  d['day'] = k + 1
  dcs += [d]
d = gezi.read_parquet('../input/eval/context.parquet')
d['day'] = 31
dcs += [d]
dc = pd.concat(dcs)

100%|██████████| 30/30 [00:28<00:00,  1.07it/s]


In [19]:
for i in tqdm(range(30)):
  dcs[i]['time'] = dcs[i].timestamp.apply(lambda x:gtime(x + 1))

100%|██████████| 30/30 [04:21<00:00, 10.41s/it]


In [20]:
dcs[29].time.max()

'3-22-0-0'

In [9]:
did = dc.did.sample(1).values[0]

In [21]:
did = '6c75a2fa6c2f4e19cc7d59c4cc6ec033'

In [22]:
did 

'6c75a2fa6c2f4e19cc7d59c4cc6ec033'

In [17]:
for i in tqdm(range(31)):
  len_ = len(dcs[i][dcs[i].did==did])
  if len_:
    print(i, len_)

 97%|█████████▋| 30/31 [00:05<00:00,  5.24it/s]

29 16


100%|██████████| 31/31 [00:05<00:00,  4.27it/s]


In [23]:
dcs[29][(dcs[29].did==did)&(dcs[29].label==1)]

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index,day,time
2478454,1,b5afeaf15ebd48084b163cdf7cba75cd,9a6e6cc222b26cc083646e13c32730ab,650894f17eee3eb5102c6394c9d9df90,f653f737068af83a709746a60da2a0ab,6c75a2fa6c2f4e19cc7d59c4cc6ec033,2694993287,0,4253243724,1426947809,2478454,30,3-21-22-23
2478458,1,b5afeaf15ebd48084b163cdf7cba75cd,9a6e6cc222b26cc083646e13c32730ab,650894f17eee3eb5102c6394c9d9df90,f653f737068af83a709746a60da2a0ab,6c75a2fa6c2f4e19cc7d59c4cc6ec033,2872596680,0,4253243724,1426937730,2478458,30,3-21-19-35


In [55]:
dcs[4][(dcs[4].did==did)&(dcs[4].label==1)]

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index,day,time
77765,1,baf32bdfc69f5b038c30085858de47ea,88315d3de949862117566caeffd81519,183e022fe835e248ec27774c2687e312,1c58433296d1b994e72747f4b3cddbf5,015675e2f6c3d05b0148d375c8525c27,3535563902,2125121607,3426259412,1424780655,77765,5,2-24-20-24
77789,1,baf32bdfc69f5b038c30085858de47ea,88315d3de949862117566caeffd81519,183e022fe835e248ec27774c2687e312,1c58433296d1b994e72747f4b3cddbf5,015675e2f6c3d05b0148d375c8525c27,2391498716,0,3426259412,1424780831,77789,5,2-24-20-27


In [24]:
dus = []
for i in tqdm(range(30)):
  d = gezi.read_parquet(f'../input/train/part_{i + 1}/user.parquet')
  d['day'] = i + 1
  dus += [d]
d = gezi.read_parquet('../input/eval/user.parquet')
d['day'] = 31
dus += [d]
du = pd.concat(dus)

100%|██████████| 30/30 [00:20<00:00,  1.32it/s]


In [32]:
dus[-1]

,did,watch,day
0,000c3ba80c2409c25397c3b7a96bf7b7,"[[1426584338, 1689874036], [1426582429, 396537...",31
1,002935918fb49444e22021a444bed3ef,"[[1427039460, 2168299795], [1427022514, 205222...",31
2,005520bbde1b22072e0f5a2a0de90307,"[[1426920166, 2338506193], [1426918447, 228034...",31
3,0128d5011e88e088c23a6ac00aa74250,"[[1426942728, 3650366537], [1426942632, 108342...",31
4,0208e13c29cb5cf22ce4c5dab56b8a5a,"[[1426955612, 1083423540], [1426955512, 119057...",31
...,...,...,...
156268,feeb8c669a7cd6f5810fecbe0c9815f1,"[[1427036845, 78776638], [1427036039, 16533837...",31
156269,feef89c4d16cf2fdfbd42f1b82ece2c7,"[[1426566573, 3853814243], [1426566205, 164466...",31
156270,ff13fa73325f60e61a2a353f9ca7018b,"[[1426860075, 247533023], [1426860038, 1817923...",31
156271,ff82b8247eaf6a9fe24235985c3215cb,"[[1426680642, 4160178488], [1426677797, 123791...",31


In [34]:
def get_dw(i, did):
  watches = dus[i][dus[i].did==did].watch.apply(json.loads).values[0]
  dw = pd.DataFrame({'timestamp':[x[0] for x in watches], 'vid': [x[1] for x in watches], 
                   'mark': [len(dcs[i][(dcs[i].did==did)&(dcs[i].vid==x[1])]) for x in watches], 
                   'time_str': [gtime(x[0]) for x in watches], 
                    'time_str2': [gtime(dcs[i][dcs[i].did==did].timestamps.values[0]) for x in watches]})
  return dw

In [35]:
def get_dc(i, did):
  return dcs[i][(dcs[i].did==did)&(dcs[i].label==1)]

In [36]:
dw = get_dw(30, '0208e13c29cb5cf22ce4c5dab56b8a5a')
dw

AttributeError: 'DataFrame' object has no attribute 'timestamps'

In [64]:
dcs[5][(dcs[5].did==did)&(dcs[5].label==1)]

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index,day,time
78016,1,baf32bdfc69f5b038c30085858de47ea,88315d3de949862117566caeffd81519,e452a790a877bea65da21f6dbe9d9265,1c58433296d1b994e72747f4b3cddbf5,015675e2f6c3d05b0148d375c8525c27,2408359345,938623579,3426259412,1424871060,78016,6,2-25-21-31


In [65]:
dw = get_dw(5, did)
dw

,timestamp,vid,mark,time_str
0,1424871576,523259998,0,2-25-21-39
1,1424871217,3951042991,0,2-25-21-33
2,1424871159,2255467444,0,2-25-21-32
3,1424780846,2391498716,0,2-24-20-27
4,1424780663,2125121607,0,2-24-20-24
5,1424738691,602531047,0,2-24-8-44
6,1424606870,3858070560,0,2-22-20-7
7,1424577963,769089035,0,2-22-12-6
8,1424561026,119656262,0,2-22-7-23
9,1424479403,3598430390,0,2-21-8-43


In [70]:
get_dc(7, did)

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index,day,time
72701,1,baf32bdfc69f5b038c30085858de47ea,88315d3de949862117566caeffd81519,8a30ae7b3ec823c36539b1dc59882fd8,1c58433296d1b994e72747f4b3cddbf5,015675e2f6c3d05b0148d375c8525c27,3091998066,2980318439,3426259412,1425027445,72701,8,2-27-16-57


In [71]:
get_dw(7, did)

,timestamp,vid,mark,time_str
0,1425038215,1385422845,0,2-27-19-56
1,1425037283,1838803242,0,2-27-19-41
2,1425027652,3160633545,0,2-27-17-0
3,1425027559,1992993790,0,2-27-16-59
4,1425027446,2980318439,0,2-27-16-57
5,1425027435,1778102039,0,2-27-16-57
6,1425027315,2818105898,0,2-27-16-55
7,1424871576,523259998,0,2-25-21-39
8,1424871217,3951042991,0,2-25-21-33
9,1424871159,2255467444,0,2-25-21-32


In [72]:
get_dw(8, did)

,timestamp,vid,mark,time_str
0,1425130391,772080798,1,2-28-21-33
1,1425130355,1640447438,0,2-28-21-32
2,1425130259,1989771416,1,2-28-21-30
3,1425130107,1803422432,1,2-28-21-28
4,1425130043,1385422845,0,2-28-21-27
5,1425125922,3589698660,0,2-28-20-18
6,1425122580,944269797,0,2-28-19-23
7,1425037283,1838803242,0,2-27-19-41
8,1425027652,3160633545,0,2-27-17-0
9,1425027559,1992993790,0,2-27-16-59


In [73]:
get_dw(11, did)

,timestamp,vid,mark,time_str
0,1425389022,96061885,0,3-3-21-23
1,1425389004,1228676722,0,3-3-21-23
2,1425388887,2566433568,1,3-3-21-21
3,1425222291,3747289313,0,3-1-23-4
4,1425222251,1778102039,0,3-1-23-4
5,1425191698,2818105898,0,3-1-14-34
6,1425130391,772080798,0,2-28-21-33
7,1425130355,1640447438,0,2-28-21-32
8,1425130259,1989771416,0,2-28-21-30
9,1425130107,1803422432,0,2-28-21-28


In [75]:
dus[-1]

,did,watch,day
0,000c3ba80c2409c25397c3b7a96bf7b7,"[[1426584338, 1689874036], [1426582429, 396537...",31
1,002935918fb49444e22021a444bed3ef,"[[1427039460, 2168299795], [1427022514, 205222...",31
2,005520bbde1b22072e0f5a2a0de90307,"[[1426920166, 2338506193], [1426918447, 228034...",31
3,0128d5011e88e088c23a6ac00aa74250,"[[1426942728, 3650366537], [1426942632, 108342...",31
4,0208e13c29cb5cf22ce4c5dab56b8a5a,"[[1426955612, 1083423540], [1426955512, 119057...",31
...,...,...,...
156268,feeb8c669a7cd6f5810fecbe0c9815f1,"[[1427036845, 78776638], [1427036039, 16533837...",31
156269,feef89c4d16cf2fdfbd42f1b82ece2c7,"[[1426566573, 3853814243], [1426566205, 164466...",31
156270,ff13fa73325f60e61a2a353f9ca7018b,"[[1426860075, 247533023], [1426860038, 1817923...",31
156271,ff82b8247eaf6a9fe24235985c3215cb,"[[1426680642, 4160178488], [1426677797, 123791...",31
